<a href="https://colab.research.google.com/github/ravimashru/100-days-of-deep-learning/blob/master/docs/days/026_Custom_Layers_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Layers in TensorFlow




In [1]:
import tensorflow as tf
from tensorflow import keras

print('TensorFlow: ', tf.__version__)
print('Keras: ', keras.__version__)

TensorFlow:  2.3.0
Keras:  2.4.0


In [2]:
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

print('Training set: ', X_train.shape, y_train.shape)
print('Validation set: ', X_valid.shape, y_valid.shape)
print('Test set: ', X_test.shape, y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
Training set:  (45000, 28, 28) (45000,)
Validation set:  (15000, 28, 28) (15000,)
Test set:  (10000, 28, 28) (10000,)


First, we will create a layer that behaves like the `tf.keras.layers.Flatten` layer and converts the `28x28` input to an array of 784 inputs.

The easiest way to create such a layer that has no weights is by using the `Lambda` layer as follows:

In [82]:
import numpy as np

flatten_layer = keras.layers.Lambda(lambda x: tf.reshape(x, [-1, np.prod(x.shape[1:])]))

Now, let us create a layer that behaves like `tf.keras.layers.Dense`. Since this layer needs to keep track of weights and biases, we can't use a simple lambda function. We need to subclass the `tf.keras.layers.Layer` class as follows:

In [93]:
class DenseLayer(keras.layers.Layer):

  # The constructor takes the following hyperparameters as arguments:
  #  - units: the number of neurons in this dense layer
  #  - activation: the activation function to be used in this layer
  def __init__(self, units, activation=None, **kwargs):
    super().__init__(**kwargs)
    self.units = units
    self.activation = keras.activations.get(activation)


  # This method is called the first time that the layer is used.
  # The variables required by the layer are created here.
  def build(self, batch_input_shape):
    self.layer_weights = self.add_weight(
        name='weights',
        shape=[batch_input_shape[-1], self.units],
        initializer='glorot_normal'
    )

    self.layer_biases = self.add_weight(name='biases', shape=[self.units], initializer='zeros')

    # Call parent's build method to notify that we are done building this layer
    super().build(batch_input_shape)


  # Method called during the forward pass
  def call(self, X):
    return self.activation(X @ self.layer_weights + self.layer_biases)
  

In case we need to create a layer that behaves differently during training and testing (e.g. `BatchNormalization` and `Dropout`), a `training` argument can be added to the `call()` function.

Let us create a dropout layer that will work during training but not testing:

In [94]:
class DropoutLayer(keras.layers.Layer):

  def __init__(self, rate=0.5, **kwargs):
    super().__init__(**kwargs)
    self.rate = rate

  def call(self, X, training=None):
    if training:
      return tf.nn.dropout(X, self.rate)
    else:
      return X
  

Now let us create a model using these custom layers:

In [97]:
input = keras.layers.Input(shape=(28,28))

# Use our custom flatten layer
flattened = flatten_layer(input)

# Use our custom dense and dropout layers
dense = DenseLayer(128, activation='relu')(flattened)
dropout = DropoutLayer(0.2)(dense)

output = DenseLayer(10, activation='softmax')(dropout)

model = keras.Model(input, output)

model.summary()

Model: "functional_72"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        [(None, 28, 28)]          0         
_________________________________________________________________
lambda_22 (Lambda)           (None, 784)               0         
_________________________________________________________________
dense_layer_9 (DenseLayer)   (None, 128)               100480    
_________________________________________________________________
dropout_layer_2 (DropoutLaye (None, 128)               0         
_________________________________________________________________
dense_layer_10 (DenseLayer)  (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


The functional API is used in this example, however the sequential API could also be used. The custom layers created behave like any other layers in Keras.

In [98]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [99]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
704/704 [==============================] - 2s 3ms/step - loss: 3.9426 - accuracy: 0.7520 - val_loss: 0.6955 - val_accuracy: 0.8491
Epoch 2/10
704/704 [==============================] - 2s 3ms/step - loss: 0.6897 - accuracy: 0.8289 - val_loss: 0.5704 - val_accuracy: 0.8754
Epoch 3/10
704/704 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.8611 - val_loss: 0.4632 - val_accuracy: 0.9005
Epoch 4/10
704/704 [==============================] - 2s 3ms/step - loss: 0.4602 - accuracy: 0.8804 - val_loss: 0.3890 - val_accuracy: 0.9203
Epoch 5/10
704/704 [==============================] - 2s 3ms/step - loss: 0.4014 - accuracy: 0.8935 - val_loss: 0.3397 - val_accuracy: 0.9247
Epoch 6/10
704/704 [==============================] - 2s 3ms/step - loss: 0.3618 - accuracy: 0.9032 - val_loss: 0.3670 - val_accuracy: 0.9250
Epoch 7/10
704/704 [==============================] - 2s 3ms/step - loss: 0.3462 - accuracy: 0.9074 - val_loss: 0.3010 - val_accuracy: 0.9337
Epoch 

In [100]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.2610 - accuracy: 0.9454


[0.2609901428222656, 0.9453999996185303]